In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By

#명시적 대기
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import time

In [5]:
import math

# 3. 통합 코드 구현

In [59]:
class ReviewInfo:
    score = ''
    title = ''
    contents = ''
    tour_year = ''
    tour_month = ''
    departure = ''
    arrival = ''
    continent = ''
    seat_class = ''
    
    def __init__(self,score,title,contents,tour_year,tour_month,departure,arrival,continent,seat_class):
        self.score = score
        self.title = title
        self.contents = contents
        self.tour_year = tour_year
        self.tour_month = tour_month 
        self.departure = departure
        self.arrival = arrival
        self.continent = continent
        self.seat_class = seat_class
        
review_list = []    

In [20]:
main_url = 'https://www.tripadvisor.co.kr/Airline_Review-d12124804-Reviews-T-Way-Airlines'
driver = webdriver.Chrome("C:/2019_join_startup/remake_digital/chromedriver_win32/chromedriver.exe")
driver.get(main_url)

In [21]:
#한국어 리뷰 수
kor_count = driver.find_element_by_xpath('/html/body/div[2]/div/div/div[6]/div/div/div/div[2]/div[2]/div[2]/div/div[1]/div[1]/div[4]/ul/li[3]/label/span[2]').text
#kor_count = driver.find_element_by_css_selector('#component_1 > div > div.flights-airline-review-page-AirlineReviewPage__contentWrapper--1vfBd > div > div > div > div.ui_container.flights-airline-review-page-AirlineReviewPage__contentCard--3SVH- > div:nth-child(2) > div:nth-child(2) > div > div.location-review-review-list-parts-ReviewFilters__filters_wrap--y1t86 > div.ui_columns > div.ui_column.is-3-tablet.is-shown-at-tablet > ul > li:nth-child(3) > label > span.location-review-review-list-parts-LanguageFilter__paren_count--2vk3f').text
kor_count = int(kor_count[:-1].split('(')[1])
kor_review_page = math.ceil(kor_count/5)

In [22]:
kor_review_page

18

# 최종 크롤링 코드

In [25]:
for page in range(1,kor_review_page+1):
    driver.find_elements_by_css_selector('.location-review-review-list-parts-ExpandableReview__cta--2mR2g')[0].click()
    try:
        #1페이지당 최대 5개 리뷰
        for j in range(0,5,1):
            #전체 데이터
            reviews =driver.find_elements_by_css_selector('.location-review-review-list-parts-SingleReview__mainCol--1hApa')

            #평가점수
            score_str = reviews[j].find_element_by_css_selector('.ui_bubble_rating').get_attribute('class')
            score = score_str.split('ui_bubble_rating bubble_')[1]

            #리뷰 제목
            title = reviews[j].find_element_by_css_selector('.location-review-review-list-parts-ReviewTitle__reviewTitle--2GO9Z').text

            #리뷰 내용
            contents = reviews[j].find_element_by_css_selector('.location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').text
            contents

            #여행 날짜(년도,월) #예외처리
            try:  
                date = reviews[j].find_element_by_css_selector(".location-review-review-list-parts-EventDate__event_date--1epHa").text.split(' ')
                date_y = date[2][:-1]
                date_m = date[3][:-1]
            except Exception as e1:
                date_y = 'Nan'
                date_m = 'Nan'
                
            #출발지 여행지 좌석
            info = reviews[j].find_element_by_css_selector('.location-review-review-list-parts-RatingLine__labelsContainer--rSajH').text.split('\n')
            #출발지
            departure = info[0].split(' - ')[0]
            #도착지
            arrival = info[0].split(' - ')[1]
            #대륙
            continent = info[1]
            #좌석
            seat_class = info[2]
            print('score:%s\n제목:%s\n내용:%s\n여행날짜(년):%s\n여행날짜(월):%s\n출발지:%s\n도착지:%s\n대륙:%s\n좌석등급:%s\n' %(score,title,contents,date_y,date_m,departure,arrival,continent,seat_class))
            print('='*100)
       
        if (page != kor_review_page+1):
            driver.find_element_by_xpath('//a[@class="ui_button nav next primary "]').click()
            time.sleep(2)
        
        print('%s 페이지 이동' %(page+1))
      
    except Exception as e:
        print('이동 오류',e)

score:50
제목:무난한 비행
내용:자주 잉요하는 항공사로 노선이 적절절해서 이용하고 있음. 항공사 직원들의 서비스가 매우 만족스러움. 자격대비 만족도 높음.
여행날짜(년):2019
여행날짜(월):10
출발지:대구
도착지:세부
대륙:동남아시아
좌석등급:이코노미

score:10
제목:뱅기 놓친다더니 ㅡ.ㅡ
내용:리뷰 읽으면서 남의 일로만 생각했는데 김.포. 공.항. 출발 40분전에 도착했는데 카운터에 직원 흔적조차 없네. 도대체 언제 카운터를 닫는 거요?!! 도심이랑 가까워서 금액도 꽤 지불하고 예약한 편도인데 놓치고 인천공항 가게 될듯. 뎅장.
여행날짜(년):2019
여행날짜(월):10
출발지:서울
도착지:타이페이
대륙:북아시아
좌석등급:이코노미

score:30
제목:좌석이 좁지만, 승무원이 친절하여 무난함.
내용:저가 항공이다보니, 좌석이 좁다.
아이들과 여행이라면 국적기 이용을 권하고 싶다.
성인들끼리라면 가성비면에서, 짧은 거리면 괜찮은편이다.
여행날짜(년):2018
여행날짜(월):11
출발지:아가나
도착지:서울
대륙:국외
좌석등급:이코노미

score:30
제목:승무원은 매우 친절. 그러나 너무 좁음
내용:항공에 큰 기대는 없었지만. 좌석이 앞뒤로 너무 좁은데 정말 아무 생각없이 뒤로 확 제끼시는 분들이 너무 많았어요.. 전 다리도 펼수없었고.. 키가 180이 넘어가시는 남자분들은 마치 의자에 딱 붙어 가는.. 우리 남편도 190이다보니 거구에.. 무릎이 그냥 앞좌석에 닿는데 자꾸 뒤로 밀어서 무릎이 멍들었어요.ㅠㅠ 좁은만큼 뒷사람이 자는지, 공간이 있는지 확인하고 넘겨줬음 좋겠어요...
여행날짜(년):2019
여행날짜(월):9
출발지:서울
도착지:하이커우
대륙:북아시아
좌석등급:이코노미

score:30
제목:좌석좁은 것빼고 무난. 하지만 하이난(싼야SYX)공항은 힘드네요
내용:좌석이 좁은것 말곤 무난합니다만.

항공사보다는 하이난(싼야SYX)공항에서 수속절차가 너무 많고, 번거롭고, 처리도 느린게 너무 힘들었네요
여행날짜(년):2019
여행날

score:10
제목:모니터 없음
내용:할말이 없습니다. 너무 열악하여서 그렇기도 하지만 탑승게이트 역시 지하철로 버스로 이동하여서 여간 불편하기 이를데 없네요.저가 항공사를 이용할수 밖에 없어서 이용하였지만 이번 출장길은 내 생애 최악이라 생각되어 다시는 저가 항공편 이용하지 않을 각오 입니다.
여행날짜(년):2019
여행날짜(월):1
출발지:서울
도착지:호치민시
대륙:동남아시아
좌석등급:이코노미

score:50
제목:저렴한 가격에 이용할 수 있는 티웨이항공.
내용:좌석이 약간 좁긴 하지만, 국내노선이나 해외단거리노선(일본) 갈 때에는 불편함을 느낄 시간도 없이 금방 착륙해서 불편함 없이 탑승할 수 있습니다. 물 한잔도 안주는 저가항공사도 있다는데, 승객 모두에게는 아니지만 원하는 승객에게 물도 주십니다. 승무원분들도 정말 친절하시고, 저렴한 가격에 이용가능해서 좋았습니다.
여행날짜(년):2019
여행날짜(월):1
출발지:무안
도착지:오이타
대륙:북아시아
좌석등급:이코노미

6 페이지 이동
score:40
제목:값싸게 이용가능한 비행
내용:왕복비행 19만원에 베트남을 다녀왔어요.
기내 baggages 도 10kg까지 넉넉해서 따로 수화물 붙일 필요없음.
다른 저가항공처럼 좁은 좌석, 비싼 기내식, 밤늦은 출발과 도착은 감수해야함.
부산 pm10시 출발해서 하노이 am1시에 도착
하노이 am2시 출발 부산am8시 도착함
여행날짜(년):2019
여행날짜(월):1
출발지:부산
도착지:하노이
대륙:동남아시아
좌석등급:이코노미

score:20
제목:화장실 청소는 승객이 해야하나?
내용:5일간의 비지니스 여행~ 구마모토까지 직항이 티웨이 항공밖에 없어서 뭐 그냥 잠깐 타는거니 타고 가자는 마음이었으나 결과부터 말하자면 "티웨이 넌 이제 완전 아웃"!
수하물 무게부터 국제선 수하물이 15KG가 뭐니? 타 항공사는 23KG인데, 지들 맘대로 15KG정해놓고, 16KG나오니까 1KG을 빼라더군요. 일반 항공사에서 훨씬 못 미치는 규정을 지들 맘대로 적용한거 아니냐고 항의

10 페이지 이동
score:30
제목:제주행 이용
내용:기내에서는 승무원 안내도 잘 받을 수 있었지만
체크인 카운터에 있던 승무원은 수화물 부칠때 우왕좌왕하니 비웃는 듯한 느낌을 주었습니다
참고 부탁드려요
여행날짜(년):2018
여행날짜(월):5
출발지:서울
도착지:제주
대륙:국내
좌석등급:이코노미

score:50
제목:티웨이 항공 세부행 이용
내용:티웨이 항공으로 대구 출발 세부행 이용해봤습니다. 돈 추가해서 최전방 유료 프리미엄 좌석을 구매해서 이용해봤는데, 일반 좌석에 비해서 여유공간이 상대적으로 편합니다.
부담 크게 안되신다면 티웨이 이용시 유료 지정 죄석 구매도 나쁘지 않네요.
여행날짜(년):2018
여행날짜(월):5
출발지:대구
도착지:세부
대륙:동남아시아
좌석등급:이코노미

score:30
제목:저가항공으로 무난합니다.
내용:저가항공사 다운 항공입니다. 좌석이 좁아서 장시간 비행으로는 허리가 아플 것 같고, 3시간 내외인 국가를 다니기 적절한 것 같습니다. 승무원들은 친절하며, 기내식은 별도로 없었습니다. 장거리보다는 근거리에 적합한 항공사입니다.
여행날짜(년):2017
여행날짜(월):7
출발지:서울
도착지:제주
대륙:국내
좌석등급:이코노미

score:50
제목:일본은 진에어 대만은 티웨이!!!
내용:저가항공은 역시 싼맛인듯 ㅎ
여행삼매경애 빠진 요즘 항공료를 아껴야 여행지에서 두배로 즐길수 있다는것!!!
서비스나 시설은 비슷하다면 포인트적립을 배제할정도로 싼가격에 표가 나왔다면 망설임없이 겟잇!!!
여기저기 다녀본결과 대만은 티웨이 일번은 진에어가 갑인듯~
그리고 언제나 내짐은 소중하기에 위탁수하물에는 조심표시 달아주는 센스~

근데 이스타랑 공동운항하지만 이스타는 비싸고 티웨이는 싼이유가 뭘까?!?!
여행날짜(년):2018
여행날짜(월):5
출발지:청주
도착지:타이페이
대륙:북아시아
좌석등급:이코노미

score:10
제목:탑승구가 제2터미널에 있었는데 제대로 안내도 안해주고 결국 비행기 놓쳤어요
내용:발권해서 표받을때 탑승권주면서 탑승구

15 페이지 이동
score:50
제목:저가항공이지만 서비스의 질이 높았습니다.
내용:가족여행인데 저가항공이라서 염려스러웠는데 티켓팅 할때 대기시간이 긴것 빼고는 모두 만족스러웠습니다. 공간이 좁은 것은 어쩔수 없겠지만 서비스가 무척 친절하고 물을 줘서 좋았습니다.
여행날짜(년):2017
여행날짜(월):11
출발지:서울
도착지:아가나
대륙:국외
좌석등급:이코노미

score:50
제목:저렴한 가격으로 여행~
내용:특가가 떠서
부모님과 누나가족이랑 대가족이 이용했음에도
메이저 항공사 2사람 정도의 가격밖에 나오지 않아서
엄청 저렴하게 잘 타고 지연되지 않고 좋은 경험을 했습니다.
다음에도 저렴한 특가가 뜬다면 다시 이용할 의시가 있습니다.
여행날짜(년):2017
여행날짜(월):9
출발지:서울
도착지:사가
대륙:북아시아
좌석등급:이코노미

score:40
제목:저렴하게 타기 좋아요
내용:연착으로 악명높았는데 거의 정시 출발했구요. 새벽 비행기가 너무 추워서 힘들긴했어요. 좌석은 저가항공에 비해 좀 넓은 편이었구요. 나름 가격대비 괜찮아서 나중에 또 특가뜨면 이용하려구요
여행날짜(년):2017
여행날짜(월):9
출발지:서울
도착지:호치민시
대륙:동남아시아
좌석등급:이코노미

score:30
제목:저렴한 LCC....
내용:저렴한 항공료로 다른 모든 것을 상쇄시키는 비행. LCC에 대한항공이나 아시아나급의 서비스를 바라는 것은 현대차를 타면서 벤츠나 BMW의 성능을 바라는 것과 같음. 가성비는 매우 좋음.
여행날짜(년):2017
여행날짜(월):1
출발지:대구
도착지:세부
대륙:동남아시아
좌석등급:이코노미

score:40
제목:실속파 여행
내용:음료서비스 없어도 제주도보다 비행기표가 쌌다. 좌석이 좀 낡은 듯 했으나 짧은 비행시간이라 괜찮았다. 올때 기념품을 많이 샀다면 일행과 나누어 짐을 싸는 게 좋겠다. 수하물 초과비용을 덜 내려면!
여행날짜(년):2017
여행날짜(월):7
출발지:서울
도착지:구마모토
대륙:북아시아
좌석등급:이코노미

16 페이지 이동
score